In [79]:
!pip install langchain
!pip install -qU langchain-groq

In [80]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("First person to land on moon was...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [81]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("api_key")
page_data=loader.load().pop().page_content
print(page_data)





















Nike Careers









































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Langua

In [82]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [83]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': '12 Months Head Coach (Senior Store Manager)',
  'experience': 'Not specified',
  'skills': 'Not specified',
  'description': 'Nike - Brisbane Airport'},
 {'role': '20 Hour Part Time Athlete (Sales Floor Associate)',
  'experience': 'Not specified',
  'skills': 'Not specified',
  'description': 'Leeds, West Yorkshire, United Kingdom'},
 {'role': '40 Hour Full Time - Athlete (Sales Floor Associate)',
  'experience': 'Not specified',
  'skills': 'Not specified',
  'description': 'Leeds, West Yorkshire, United Kingdom'},
 {'role': '8 Hour Part Time - Athlete (Sales Floor Associate)',
  'experience': 'Not specified',
  'skills': 'Not specified',
  'description': 'Leeds, West Yorkshire, United Kingdom'},
 {'role': 'Abteilungsleiter/in (Coach) - 37.5 Std. - NFS Wolfsburg (m/w/d)',
  'experience': 'Not specified',
  'skills': 'Not specified',
  'description': 'Wolfsburg, Lower Saxony, Germany'},
 {'role': 'Account Executive Indonesia, SEA&I',
  'experience': 'Not specified',
  'skil

In [84]:
type(json_res)

list

In [85]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [88]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df 

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [89]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [90]:
jobs = json_res if isinstance(json_res, list) else [json_res]

for job in jobs:
    links = collection.query(
        query_texts=job["skills"],
        n_results=2
    )["metadatas"]
  


In [91]:
print(type(job))

<class 'dict'>


In [92]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [93]:
job

{'role': 'Allocator, SEA&I',
 'experience': 'Not specified',
 'skills': 'Not specified',
 'description': 'South, Singapore, South East, Singapore'}

In [94]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Nithya, a business development executive at ABC. ABC is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of ABC 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase ABC's portfolio: {link_list}
        Remember you are Nithya, BDE at ABC. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Allocation Solutions for SEA&I Role with ABC

Dear Hiring Manager,

I came across the Allocator, SEA&I role at your esteemed organization and was impressed by the opportunity to contribute to your team's success. As a Business Development Executive at ABC, an AI & Software Consulting company, I'd like to introduce our expertise in facilitating seamless integration of business processes through automated tools.

At ABC, we have a proven track record of empowering enterprises with tailored solutions, resulting in scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts can help you streamline allocation processes, leveraging cutting-edge technologies to drive business growth.

I'd like to highlight some of our notable projects that demonstrate our capabilities:
- Our Kotlin backend portfolio (https://example.com/kotlin-backend-portfolio) showcases our proficiency in developing robust and scalable backend solutions, which ca